## PyLog HLS C Code Generation

### Preparation
Set PyLog path and import PyLog. 

In [ ]:
import sys
sys.path.extend(['/home/shuang91/pylog/'])

import numpy as np
from pylog import *

### Subscripts
PyLog supports Python subscripts with arbitrary experssions. 

To annotate a Python function for compilation, simply decorate the function with `@pylog` decorator. 

In [ ]:
@pylog
def pl_subscript(a, b):
    for i in range(4, 15, 2):
        a[b[i]][f(a * b + 13)][csdf*w34 - 1] = b
    return c

Call the decorated Python function with sample input will compile the function:  

In [ ]:
a = np.array([1, 3, 6, 7, 10])
b = np.array([1, 3, 6, 7, 10])
c = pl_subscript(a, b)

Now let's take a look at the generated HLS C code: 

In [ ]:
!pygmentize /home/shuang91/vivado_projects/pylog_projects/pl_subscript/pl_subscript.cpp

### Control Statements
PyLog supports Python control statements, e.g. `if elif else`, `for`, `while`, etc. PyLog also supports using `range` to specify boundaries of a `for` loop. 

In [ ]:
@pylog
def pl_control(a, b):

    def func_inside(c):
        return c + 1

    a = func_inside(another_func(b))

    for ii in range(100, 20, -4):
        for jj in range(10, 245, 3):
            a = b+c

    while (a > 100):
        if (a < foo(b)):
            test += 1
        elif (b > c):
            join -= -9
            asdf = swr34cv_1 + 1
        else:
            return 0
        c = (foo(a + b, c*d)) if (a > 0) else res
        c = 100 if (a > 0) else -10
    return c

Compile the decorated function: 

In [ ]:
a = np.array([1, 3, 6, 7, 10])
b = np.array([1, 3, 6, 7, 10])
c = pl_control(a, b)

Inspect the generated HLS C code:

In [ ]:
!pygmentize /home/shuang91/vivado_projects/pylog_projects/pl_control/pl_control.cpp

### Example: Matmul
Let's take a look at a simple matrix multiplication in PyLog.

In [ ]:
@pylog
def pl_matmul(a, b, c, d):
    
    buf = np.empty([16, 16], int)
    pragma("HLS array_partition variable=buf")

    def matmul(a, b, c):
        for i in range(32):
            for j in range(32).unroll(4):
                tmp = 0.
                for k in range(32).pipeline():
                    tmp += a[i][k] * b[k][j]
                c[i][j] = tmp


    def vecadd(a, b, c):
        for i in range(32):
            c[i] = a[i] + b[i]


    matmul(a, b, c)

    return 0

In [ ]:
length = 1024
a = np.random.rand(length, length)
b = np.random.rand(length, length)
c = np.zeros((length, length))
d = np.random.rand(1)
pl_matmul(a, b, c, d)

Generated HLS C code:

In [ ]:
!pygmentize /home/shuang91/vivado_projects/pylog_projects/pl_matmul/pl_matmul.cpp

### Arbitrary Bitwidth Data Types
PyLog allows users to use arbitrary bitwidth integers and fixed-point numbers. As an example, let's change the inputs to the above `pl_matmul` function to fixed-point numbers. `pl_fixed(8, 3)` is the 8-bit fixed-point format with 3 bits above decimal point.

In [ ]:
length = 1024
a = np.random.rand(length, length).astype(pl_fixed(8, 3))
b = np.random.rand(length, length).astype(pl_fixed(8, 3))
c = np.zeros((length, length)).astype(pl_fixed(16, 6))
d = np.random.rand(1).astype(np.int8)
pl_matmul(a, b, c, d)

In [ ]:
!pygmentize /home/shuang91/vivado_projects/pylog_projects/pl_matmul/pl_matmul.cpp

### Python Simulation
Similar to "CSim" in C-based HLS, PyLog support Python-based functionality simulation, `pysim`. PySim can be enabled by setting the PyLog mode to `'pysim'`. In PySim mode, when the decorated function is called, instead of going through PyLog compilation, PyLog will run the function as it is with standard Python interpreter. 

In [ ]:
from pysim import *

In [ ]:
@pylog(mode='pysim')
def pl_top(a, b, c, d):
    
    buf = np.empty([16, 16], int)
    pragma("HLS array_partition variable=buf")

    def matmul(a, b, c):
        for i in range(32):
            for j in range(32).unroll(4):
                tmp = 0.
                for k in range(32).pipeline():
                    tmp += a[i][k] * b[k][j]
                c[i][j] = tmp


    def vecadd(a, b, c):
        for i in range(32):
            c[i] = a[i] + b[i]


    matmul(a, b, c)

    return 0

In [ ]:
length = 32
a = np.random.rand(length, length)
b = np.random.rand(length, length)
c = np.zeros((length, length))
d = np.random.rand(1)
pl_top(a, b, c, d)
print(c)